In [7]:
import csv
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
from os import path

os.getcwd()

'/home/matta/recipy_maker/Web_Scraping'

In [58]:
MAIN_STRING = "carouselNav__list elementFont__resetList recipeCarousel__list"  # class for main recipe category page navlist
CAT_STRING = "category-page-list-content category-page-list-content__recipe-card karma-main-column"  # Class for category page div
CARDS_STRING = "card__titleLink manual-link-behavior elementFont__title margin-8-bottom"  # Class for most popular recipe cards


def get_links():
    if path.exists('list_w_img.csv'):
        df = pd.read_csv('list.csv')
    else:
        source = requests.get('https://www.allrecipes.com/recipes/').text
        soup = BeautifulSoup(source, 'lxml')
        recipe_list = soup.find('ul', class_=MAIN_STRING)

        links = []
        pop_recipes = []
        for a in recipe_list.find_all('a', href=True):
            links.append(a['href'])
            for i, _ in enumerate(links):
                recipe_category = requests.get(links[i]).text
                cat_soup = BeautifulSoup(recipe_category, 'lxml')  # category page
                most_popular = cat_soup.find('div', class_=CAT_STRING)
                for a in most_popular.find_all('a', class_=CARDS_STRING, href=True):
                    if re.search('https://www.allrecipes.com/recipe/.*', a['href']):
                        pop_recipes.append([a['href'], a.div['data-src']])



        df = pd.DataFrame(pop_recipes, columns=["link","image_url"])
        df.to_csv('list_w_img.csv', index=False)
    return df

In [57]:
# l = 'https://www.allrecipes.com/recipes/76/appetizers-and-snacks/'

# recipe_category = requests.get(l).text
# cat_soup = BeautifulSoup(recipe_category, 'lxml')  # category page
# most_popular = cat_soup.find('div', class_=CAT_STRING)

# #print(most_popular)
# for a in most_popular.find_all('a', class_=CARDS_STRING,href=True):
#     if re.search('https://www.allrecipes.com/recipe/.*', a['href']):
#         print(a['href'])
#         print(a.div['data-src'])

In [59]:
df = get_links()

link_array = df['link'].to_numpy()


In [ ]:
link_array

In [109]:
recipe_info = []
for link in link_array:
    source = requests.get(link).text
    soup = BeautifulSoup(source, 'lxml')

    title = soup.find('h1').text
    description = soup.find('p').text
    rating = soup.find('span',class_="review-star-text visually-hidden").text

    ingredient_list = soup.find('ul',class_ = "ingredients-section")
    ingredients = []
    for ingredient in ingredient_list.find_all('span',class_="ingredients-item-name elementFont__body"):
        ingredients.append(ingredient.text)

    instruction_list = soup.find('ul',class_ = "instructions-section")
    instructions = []
    for instruction in instruction_list.find_all('p'):
        instructions.append(instruction.text)
    
    recipe_info.append([link,title,description,rating,ingredients,instructions])



In [163]:
df2 = pd.DataFrame(recipe_info)

recipe_data = recipe_data.rename(columns={0:'Link',1:'Title',2:'Description',3:'Rating',4:'Ingredients',5:'Instructions'})
recipe_data.to_pickle('full_data.pkl')

In [165]:
recipe_data

,Link,Title,Description,Rating,Ingredients,Instructions
0,https://www.allrecipes.com/recipe/221286/tradi...,Traditional Mexican Guacamole,This guacamole is great! Whether or not you wa...,Rating: 4.5 stars,"[2 avocados, peeled and pitted , 1 cup chopped...","[Mash avocados in a bowl until creamy., Mix to..."
1,https://www.allrecipes.com/recipe/14811/creamy...,Creamy Dill Cucumber Toasties,I got this recipe years ago from a friend of a...,Rating: 4.5 stars,"[1 (8 ounce) package cream cheese, softened , ...","[In a medium bowl, mix together cream cheese, ..."
2,https://www.allrecipes.com/recipe/278236/buffa...,Buffalo Chicken Wings in a Jar,This fun-sized take on the game day appetizer ...,Rating: 4.5 stars,"[1 ½ pounds bone-in, skin-on chicken thighs , ...",[Heat a dry saucepan over high heat. Place thi...
3,https://www.allrecipes.com/recipe/223491/fully...,Fully Loaded Deviled Eggs,My children and husband go nuts over these. I ...,Rating: 4.5 stars,"[12 eggs , 1 teaspoon vegetable oil , 1 small ...",[Place eggs into a large pot and cover with li...
4,https://www.allrecipes.com/recipe/69225/strawb...,Strawberry Bruschetta,This is a delicious variation of the popular t...,Rating: 4.5 stars,"[24 slices French baguette , 1 tablespoon butt...",[Preheat your oven's broiler. Spread a thin la...
...,...,...,...,...,...,...
2334,https://www.allrecipes.com/recipe/51283/maple-...,Maple Salmon,This maple glazed salmon is delicious and very...,Rating: 4.5 stars,"[¼ cup maple syrup , 2 tablespoons soy sauce ,...","[Stir together maple syrup, soy sauce, garlic,..."
2335,https://www.allrecipes.com/recipe/50648/feta-e...,Feta Eggs,A very TASTY way to add zip to boring scramble...,Rating: 4.5 stars,"[1 tablespoon butter , ¼ cup chopped onion , 4...",[Melt butter in a skillet over medium heat. Sa...
2336,https://www.allrecipes.com/recipe/244553/chef-...,Chef John's Baba Ghanoush,"No matter what you're grilling this summer, ch...",Rating: 4.5 stars,"[4 large Italian eggplants , 2 cloves crushed ...",[Preheat an outdoor grill for medium-high heat...
2337,https://www.allrecipes.com/recipe/222352/jamie...,Jamie's Sweet and Easy Corn on the Cob,My mother-in-law gave me this quick and easy r...,Rating: 5 stars,"[2 tablespoons white sugar , 1 tablespoon lemo...",[Fill a large pot about 3/4 full of water and ...
